# QAOA Circuit: Example Run

This notebook demonstrates how to use the `SimQC-TTN` package to simulate a QAOA circuit interactively

## 1. Import required modules

In [ ]:
# Setup: add src/ to sys.path
# Must !!! This allows importing from src/ even when running the notebook from notebooks/ directory !!!
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src = os.path.join(repo_root, "src")
if src not in sys.path:
    sys.path.insert(0, src)
    

In [ ]:
import numpy as np
from simqc_ttn.simulation.sim_qaoa_circ import sim_qaoa_flexible,sim_qaoa_native,generate_linear_graph,generate_3_regular_graph,chi_bra_layers
import matplotlib.pyplot as plt

## 1) Configure circuit

In [ ]:
# Number of qubits
num_of_qbts = 16 

# Bond dimensions for the variational binary tree tensor network layers
# Example: chi_bra = [8, 4, 2] for a 8-qubit system, which corresponds to 3 layers of the binary tree (2^3 = 8)
# Root node:  R
#            / \ -----> chi_bra[0] = 8
#           L1   R1
#          / \   / \ -----> chi_bra[1] = 4
#        L2  L3 R2  R3
#        /\  /\ /\  /\ -----> chi_bra[2] = 2
#      q1q2q3q4q5q6q7q8
chi_bra =chi_bra_layers([12,8,4,2]) # Bond dimensions for the variational binary tree tensor network layers

# Circuit depth: number of QAOA layers (Cost function + Mixer)
# For standard QAOA, one depth = one layer of cost Hamiltonian + one layer of mixer Hamiltonian

circ_depth =4

# Number of sweeps for the DMRG optimization
num_sweeps = 4

# Generate a random 3-regular graph for the QAOA cost Hamiltonian
seed = 42  # Random seed for reproducibility
graph_obj,edge_list = generate_3_regular_graph(num_of_qbts,seed=seed) # List of edges in the graph, to be used in the cost Hamiltonian

## 2) Standard Run: One depth = One layer of full QAOA (Cost function + Mixer)

In [ ]:
dmrg_circ, layer_fids = sim_qaoa_native(edges=edge_list,
                                            circ_depth=circ_depth,
                                            num_qbts=num_of_qbts,
                                            chi_bra=chi_bra,
                                            sweep_per_comp=num_sweeps)
    

F_tilde = np.prod(layer_fids)
print('F_tilde: ',F_tilde)

## 3) Flexible Run: A single depth is determined by the number of two-qubit gates considered

In [ ]:
# For flexible QAOA (see below), one depth still the same (Cost function + Mixer) but compression layer is determined by the number of two-qubit gates considered
N2g_per_comp = 20 # Number of two-qubit gates per compression step, must be less than total number of edges in the graph
dmrg_circ, layer_fids = sim_qaoa_flexible(edges=edge_list,
                                               num_QAOA_layer=circ_depth,
                                               num_qbts=num_of_qbts,
                                               chi_bra=chi_bra,
                                               N2g_per_comp=N2g_per_comp,
                                               sweep_per_comp=num_sweeps)
    
    
F_tilde = np.prod(layer_fids)
print('F_tilde: ',F_tilde)